In [33]:
from flask import Flask, render_template_string, jsonify, request
import plotly.graph_objs as go
from threading import Thread, Event

app = Flask(__name__)
stop_event = Event()

# Define your large dataset of integers here
large_dataset = list(range(1, 759))

@app.route('/')
def index():
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <title>Plotly in Flask</title>
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                .container {
                    display: flex;
                    justify-content: space-around;
                }
                .left-column, .right-column {
                    width: 45%;
                    margin-bottom: 20px;
                }
                .plot-container {
                    width: 100%;
                }
                .plot {
                    width: 100%;
                    margin-bottom: 20px;
                }
                table {
                    width: 100%;
                    border-collapse: collapse;
                    margin-bottom: 20px;
                }
                th, td {
                    border: 1px solid black;
                    padding: 8px;
                    text-align: left;
                }
                th {
                    background-color: #f2f2f2;
                }
            </style>
        </head>
        <body>
            <h1>Plotly Plots and Data Table</h1>
            <div class="container">
                <div class="left-column">
                    <h2>Data Table</h2>
                    <table>
                        <tr>
                            <th>Header 1</th>
                            <th>Header 2</th>
                        </tr>
                        <tr>
                            <td>Data 1</td>
                            <td>Data 2</td>
                        </tr>
                        <tr>
                            <td>Data 3</td>
                            <td>Data 4</td>
                        </tr>
                        <!-- Add more rows as needed -->
                    </table>
                </div>
                <div class="right-column">
                    <div class="plot-container">
                        <div class="plot">
                            <label for="datasetInput1">Enter Dataset Number 1:</label>
                            <input type="text" id="datasetInput1">
                            <button onclick="submitDataset(1)">Plot</button>
                            <div id="plot1"></div>
                        </div>
                        <div class="plot">
                            <label for="datasetInput2">Enter Dataset Number 2:</label>
                            <input type="text" id="datasetInput2">
                            <button onclick="submitDataset(2)">Plot</button>
                            <div id="plot2"></div>
                        </div>
                        <div class="plot">
                            <div id="plot3"></div>
                        </div>
                    </div>
                </div>
            </div>
            <script>
                var graphDiv1 = document.getElementById('plot1');
                var graphDiv2 = document.getElementById('plot2');
                var graphDiv3 = document.getElementById('plot3');
                var hasPlot1 = false;
                var hasPlot2 = false;
                var hasPlot3 = false;

                function fetchPlot(dataset, plotDiv) {
                    fetch(`/plot?dataset=${dataset}`)
                        .then(response => response.json())
                        .then(data => {
                            var parsedData = JSON.parse(data);
                            if (plotDiv === 'plot1') {
                                if (hasPlot1) {
                                    Plotly.react(graphDiv1, parsedData.data, parsedData.layout);
                                } else {
                                    Plotly.newPlot(graphDiv1, parsedData.data, parsedData.layout);
                                    hasPlot1 = true;
                                }
                            } else if (plotDiv === 'plot2') {
                                if (hasPlot2) {
                                    Plotly.react(graphDiv2, parsedData.data, parsedData.layout);
                                } else {
                                    Plotly.newPlot(graphDiv2, parsedData.data, parsedData.layout);
                                    hasPlot2 = true;
                                }
                            }
                        })
                        .catch(error => console.error('Error:', error));
                }

                function fetchPlotSum(sumValue) {
                    fetch(`/plot_sum?sum=${sumValue}`)
                        .then(response => response.json())
                        .then(data => {
                            var parsedData = JSON.parse(data);
                            if (hasPlot3) {
                                Plotly.react(graphDiv3, parsedData.data, parsedData.layout);
                            } else {
                                Plotly.newPlot(graphDiv3, parsedData.data, parsedData.layout);
                                hasPlot3 = true;
                            }
                        })
                        .catch(error => console.error('Error:', error));
                }

                function submitDataset(dataset) {
                    var datasetValue = parseFloat(document.getElementById(`datasetInput${dataset}`).value) || 0;
                    var plotDiv = `plot${dataset}`;
                    fetchPlot(datasetValue, plotDiv);
                    if (dataset === 1 || dataset === 2) {
                        updateCombinedPlot();
                    }
                }

                function updateCombinedPlot() {
                    var dataset1 = parseFloat(document.getElementById('datasetInput1').value) || 0;
                    var dataset2 = parseFloat(document.getElementById('datasetInput2').value) || 0;
                    var sum = dataset1 + dataset2;
                    fetchPlotSum(sum);
                }

                // Initial plots (optional)
                // fetchPlot(1, 'plot1');
                // fetchPlot(2, 'plot2');
            </script>
        </body>
        </html>
    ''')

@app.route('/plot')
def plot():
    dataset = request.args.get('dataset', '1')  # Ensure dataset is a string
    try:
        dataset = int(dataset)
    except ValueError:
        dataset = 1  # Default to dataset 1 if input is not a valid integer
    
    if dataset in large_dataset:
        data = [go.Bar(y=[dataset])]
    else:
        data = [go.Bar(y=[0])]  # Default plot if dataset not found

    layout = go.Layout(title=f'Plot for Dataset {dataset}')
    fig = go.Figure(data=data, layout=layout)
    graphJSON = fig.to_json()  # Convert the figure to JSON format
    return jsonify(graphJSON)

@app.route('/plot_sum')
def plot_sum():
    sum_value = float(request.args.get('sum', '0'))
    data = [go.Scatter(x=[1, 2, 3], y=[sum_value, sum_value * 2, sum_value * 3], mode='lines')]
    layout = go.Layout(title=f'Plot for Sum: {sum_value}')
    fig = go.Figure(data=data, layout=layout)
    graphJSON = fig.to_json()  # Convert the figure to JSON format
    return jsonify(graphJSON)

@app.route('/stop')
def stop():
    stop_event.set()
    return 'Server stopping...'

def run_app():
    app.run(port=5000, use_reloader=False)

In [34]:
run_app()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Jun/2024 22:31:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:31:12] "GET /plot?dataset=3 HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:31:12] "GET /plot_sum?sum=3 HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:31:14] "GET /plot?dataset=5 HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2024 22:31:14] "GET /plot_sum?sum=8 HTTP/1.1" 200 -
